In [1]:
from symred.chem import xyz_from_pubchem
import numpy as np
from openfermion import FermionOperator
from scipy.sparse.linalg import expm
from openfermion import get_sparse_operator
from symred.chem import Draw_molecule

In [2]:
xyz_file = '4\n \nH\t0\t0\t0\nH\t2.45366053071732\t0\t0\nH\t2.45366053071732\t2.45366053071732\t0\nH\t0\t2.45366053071732\t0'

viewer = Draw_molecule(xyz_file,
                       width=400,
                       height=400,
                       style="sphere")
viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
print(xyz_file)

4
 
H	0	0	0
H	2.45366053071732	0	0
H	2.45366053071732	2.45366053071732	0
H	0	2.45366053071732	0


In [4]:
from symred.chem import PySCFDriver

In [5]:
basis = 'sto-3g' # 'STO-3G'#
convergence = 1e-6
charge= 0
max_hf_cycles=50
ram = 8_000
run_mp2  = False
run_cisd = False
run_ccsd = True
run_fci  = False

In [6]:
pyscf_obj = PySCFDriver(xyz_file,
                       basis,
                       convergence=convergence,
                       charge=charge,
                       max_ram_memory=ram,
                       max_hf_cycles=max_hf_cycles,
                       
                       run_mp2=run_mp2,
                       run_cisd=run_cisd,
                       run_ccsd=run_ccsd,
                       run_fci=run_fci)

In [7]:
pyscf_obj.run_pyscf()

In [8]:
pyscf_obj.pyscf_hf.converged

True

In [9]:
pyscf_obj.pyscf_ccsd.converged

True

In [10]:
# T1 = pyscf_obj.pyscf_ccsd.t1
# T2 = pyscf_obj.pyscf_ccsd.t2

# from pyscf import cc
# ccsd = cc.CCSD(pyscf_obj.pyscf_hf)
# ccsd.kernel(t1= T1, t2=T2)
# ccsd.converged


In [11]:
2*pyscf_obj.pyscf_hf.mol.nao

8

In [12]:
from symred.chem import FermionicHamilt

In [13]:
H_ferm = FermionicHamilt(pyscf_obj.pyscf_hf)

H_ferm.build_operator()

H_ferm.fermionic_molecular_hamiltonian

() 1.1676751516332355
((0, 1), (0, 0)) -1.0941167141762014
((0, 1), (6, 0)) -0.02518721447269935
((1, 1), (1, 0)) -1.0941167141762014
((1, 1), (7, 0)) -0.02518721447269935
((2, 1), (2, 0)) -1.045835726765513
((3, 1), (3, 0)) -1.045835726765513
((4, 1), (4, 0)) -1.0458357267655125
((5, 1), (5, 0)) -1.0458357267655125
((6, 1), (0, 0)) -0.025187214472699386
((6, 1), (6, 0)) -1.0047159790974518
((7, 1), (1, 0)) -0.025187214472699386
((7, 1), (7, 0)) -1.0047159790974518
((0, 1), (0, 1), (0, 0), (0, 0)) 0.18176822902112627
((0, 1), (0, 1), (0, 0), (6, 0)) -0.027327679168640887
((0, 1), (0, 1), (2, 0), (2, 0)) 0.03788747346210273
((0, 1), (0, 1), (4, 0), (4, 0)) 0.1144833788422987
((0, 1), (0, 1), (6, 0), (0, 0)) -0.027327679168640897
((0, 1), (0, 1), (6, 0), (6, 0)) 0.04713286796054257
((0, 1), (1, 1), (1, 0), (0, 0)) 0.18176822902112627
((0, 1), (1, 1), (1, 0), (6, 0)) -0.027327679168640887
((0, 1), (1, 1), (3, 0), (2, 0)) 0.03788747346210273
((0, 1), (1, 1), (5, 0), (4, 0)) 0.1144833788422

In [14]:
hf_state = H_ferm.hf_comp_basis_state
hf_state


array([1, 1, 1, 1, 0, 0, 0, 0])

In [15]:
hf_ket = H_ferm.hf_ket
H_mat = H_ferm.get_sparse_ham()

In [16]:
from pyscf.cc.addons import spatial2spin

# doubles!

t2 = spatial2spin(pyscf_obj.pyscf_ccsd.t2)
no, nv = t2.shape[1:3]
nmo = no + nv
double_amps = np.zeros((nmo, nmo, nmo, nmo))
double_amps[no:,:no,no:,:no] = .5 * t2.transpose(2,0,3,1)

In [17]:
# singles

t1 = spatial2spin(pyscf_obj.pyscf_ccsd.t1)
no, nv = t1.shape
nmo = no + nv
ccsd_single_amps = np.zeros((nmo, nmo))
ccsd_single_amps[no:,:no] = t1.T
ccsd_single_amps.shape

(8, 8)

In [18]:
double_amplitudes_list=[]
double_amplitudes = double_amps
for i, j, k, l in zip(*double_amplitudes.nonzero()):
    if not np.isclose(double_amplitudes[i, j, k, l], 0):
        double_amplitudes_list.append([[i, j, k, l],
                                       double_amplitudes[i, j, k, l]])
    
double_amplitudes_list

[[[4, 0, 5, 1], -0.5456412208879349],
 [[4, 0, 6, 2], 2.208863578892679e-08],
 [[4, 0, 7, 3], -1.7451411549163962e-08],
 [[4, 1, 5, 0], 0.5456412208879349],
 [[4, 1, 7, 2], 3.954004733809075e-08],
 [[4, 2, 5, 3], 0.0366321093272368],
 [[4, 2, 6, 0], -2.208863578892679e-08],
 [[4, 2, 7, 1], -3.954004733809075e-08],
 [[4, 3, 5, 2], -0.0366321093272368],
 [[4, 3, 7, 0], 1.7451411549163962e-08],
 [[5, 0, 4, 1], 0.5456412208879349],
 [[5, 0, 6, 3], 3.954004733809075e-08],
 [[5, 1, 4, 0], -0.5456412208879349],
 [[5, 1, 6, 2], -1.7451411549163962e-08],
 [[5, 1, 7, 3], 2.208863578892679e-08],
 [[5, 2, 4, 3], -0.0366321093272368],
 [[5, 2, 6, 1], 1.7451411549163962e-08],
 [[5, 3, 4, 2], 0.0366321093272368],
 [[5, 3, 6, 0], -3.954004733809075e-08],
 [[5, 3, 7, 1], -2.208863578892679e-08],
 [[6, 0, 4, 2], -2.208863578892679e-08],
 [[6, 0, 5, 3], -3.954004733809075e-08],
 [[6, 0, 7, 1], 0.0006616412681631195],
 [[6, 1, 5, 2], 1.7451411549163962e-08],
 [[6, 1, 7, 0], -0.0006616412681631195],
 [[6, 

In [19]:
single_amplitudes_list = []
for i, j in zip(*ccsd_single_amps.nonzero()):
    single_amplitudes_list.append([[i, j], ccsd_single_amps[i, j]])

single_amplitudes_list

[[[4, 0], -3.912181678103236e-15],
 [[4, 2], -6.110293692729721e-08],
 [[5, 1], -3.912181678103236e-15],
 [[5, 3], -6.110293692729721e-08],
 [[6, 0], -0.4435278092467373],
 [[6, 2], 4.3248127744901715e-15],
 [[7, 1], -0.4435278092467373],
 [[7, 3], 4.3248127744901715e-15]]

In [20]:
generator_t2 = FermionOperator()

# Add double excitations
for (i, j, k, l), t_ijkl in double_amplitudes_list:
    i, j, k, l = int(i), int(j), int(k), int(l)
    generator_t2 += FermionOperator(((i, 1), (j, 0), (k, 1), (l, 0)), t_ijkl)
#     if anti_hermitian:
#         generator += FermionOperator(((l, 1), (k, 0), (j, 1), (i, 0)),
#                                      -t_ijkl)

In [21]:
generator_t1 = FermionOperator()
for (i, j), t_ij in single_amplitudes_list:
    i, j = int(i), int(j)
    generator_t1 += FermionOperator(((i, 1), (j, 0)), t_ij)

In [22]:
T = generator_t1 + 0.5*generator_t2

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))

In [96]:
ccsd_ket =  ccsd_ansatz @ hf_ket
hf_ket.conj().T @ H_mat @ ccsd_ket

(-1.8261563137028727+0j)

In [97]:
pyscf_obj.pyscf_ccsd.e_tot 

-1.8261563137028722

In [98]:
# intermediate normalization
hf_ket.conj().T  @ ccsd_ket


(1+0j)

In [99]:
normalized_state = ccsd_ket / np.sqrt(ccsd_ket.conj().T@ccsd_ket)

E_from_vec = normalized_state.conj().T @ H_mat @ normalized_state
E_from_vec

(-1.7418931709154082+0j)

In [100]:
abs(pyscf_obj.pyscf_ccsd.e_tot - E_from_vec)

0.08426314278746405

In [86]:
for ind, amp in enumerate(normalized_state):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(0.5276099995527699+0j)|00001111>
(-1.8433902498537798e-08+0j)|00011011>
(1.8433902498537798e-08+0j)|00100111>
(0.08576206597664797+0j)|00110011>
(-0.4725840345446496+0j)|00111100>
(0.014071962063256225+0j)|01001110>
(2.250980991942932e-08+0j)|01100110>
(0.19207141387290994+0j)|01110010>
(-0.014071962063256225+0j)|10001101>
(2.2509809919429316e-08+0j)|10011001>
(-0.19207141387290996+0j)|10110001>
(-0.47771932255267385+0j)|11000011>
(0.03172735005536297+0j)|11001100>
(2.6460860497891207e-08+0j)|11011000>
(-2.6460860497891207e-08+0j)|11100100>
(0.43305382406373405+0j)|11110000>


In [87]:
trotter_op = np.eye(2**H_ferm.n_qubits)
for op in list(generator_t2):
    trotter_op*= expm(get_sparse_operator(op, n_qubits=H_ferm.n_qubits))

    
ccsd_trotter = trotter_op @ hf_ket


normalized_trotter = ccsd_trotter / np.sqrt(ccsd_trotter.conj().T@ccsd_trotter)

normalized_trotter.conj().T @ H_mat @ normalized_trotter

(-1.5162206379375986+0j)

In [88]:
for ind, amp in enumerate(normalized_trotter):
    if not np.isclose(amp, 0):
        binary_string = np.binary_repr(ind, width=H_ferm.n_qubits)
        print(f'{amp}|{binary_string}>')

(0.8277926697224702+0j)|00001111>
(0.0004549263477769256+0j)|00110011>
(-0.3751679040580851+0j)|00111100>
(-1.5187538762098153e-08+0j)|01011010>
(2.7186649612082093e-08+0j)|01100110>
(-1.199911084998394e-08+0j)|01101001>
(-1.199911084998394e-08+0j)|10010110>
(2.7186649612082093e-08+0j)|10011001>
(-1.5187538762098153e-08+0j)|10100101>
(-0.3792446292495347+0j)|11000011>
(0.025187231373688078+0j)|11001100>
(0.1718931276157827+0j)|11110000>


In [89]:
ccsd_ansatz.shape

(256, 256)

In [90]:
zero_state = np.zeros((H_mat.shape[0],1))
zero_state[0]=1

# T = generator_t1 + 0.5*generator_t2
hf_ket.conj().T @ np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat @ ccsd_ansatz @ hf_ket

matrix([[-1.82615631+0.j]])

In [33]:
vals, vecs = np.linalg.eigh(H_mat.todense())
min(vals)

-1.8743108952541805

In [34]:
H_rot =  np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat.todense() @ ccsd_ansatz 
# H_rot =  H_mat.todense() @ ccsd_ansatz 
# H_rot = np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat.todense() 

vals, vecs = np.linalg.eigh(H_rot)
min(vals)

-2.096006827134742

In [40]:
ccsd_ansatz_inv = expm(get_sparse_operator(-1*T, n_qubits=H_ferm.n_qubits))
H_rot =  ccsd_ansatz_inv @ H_mat @ ccsd_ansatz 

vals, vecs = np.linalg.eigh(H_rot.todense())
print(min(vals))

-2.0960068271347416


In [41]:
hf_ket.conj().T @ H_rot @ hf_ket

(-1.826156313702873+0j)

In [77]:
from symred.symplectic_form import PauliwordOp
conv_dic = {0: 'I', 1:'X'}
P = ''.join([conv_dic[i] for i in H_ferm.hf_comp_basis_state])
HF_ansatz = PauliwordOp([P],[1])
print(HF_ansatz)

HF_ans_mat = HF_ansatz.to_sparse_matrix

1.0000000000+0.0000000000j XXXXIIII


In [66]:
np.allclose(HF_ans_mat.conj().T.todense(),  HF_ans_mat.todense())

True

In [75]:
np.allclose(HF_ans_mat@zero_state, hf_ket.reshape(-1,1))

True

In [76]:
# H_rot =  HF_ans_mat.conj().T @ ccsd_ansatz_inv @ H_mat @ ccsd_ansatz @ HF_ans_mat
# vals, vecs = np.linalg.eigh(H_rot.todense())
# print(min(vals))


fin_state = ccsd_ansatz @ HF_ans_mat@zero_state
fin_state.conj().T @ H_mat @ fin_state 


array([[-9.28833265+0.j]])

In [82]:
zero_state = np.zeros((H_mat.shape[0],1))
zero_state[0]=1

zero_state.conj().T @HF_ans_mat.conj().T @ np.linalg.pinv(ccsd_ansatz.todense()) @ H_mat @ ccsd_ansatz @ HF_ans_mat@ zero_state

matrix([[-1.82615631+0.j]])

In [83]:
zero_state.conj().T @HF_ans_mat.conj().T @ ccsd_ansatz.todense().conj().T @ H_mat @ ccsd_ansatz @ HF_ans_mat@ zero_state



matrix([[-9.28833265+0.j]])

In [130]:
state1 = ccsd_ansatz @ HF_ans_mat@ zero_state

state2 = state1.conj().T

state3 = zero_state.conj().T @HF_ans_mat.conj().T @ np.linalg.pinv(ccsd_ansatz.todense())

print(np.allclose(state2,
           state3))

state3 @ state1

False


matrix([[1.+0.j]])

In [ ]:
from openfermion import hermitian_conjugated
T = generator_t1 + 0.5*generator_t2

T_dagger = hermitian_conjugated(T)

ccsd_ansatz = expm(get_sparse_operator(T, n_qubits=H_ferm.n_qubits))